In [2]:
import pyvi
import re
import json
from pyvi import ViTokenizer

In [5]:
class DataPreprocessing():
    def __init__(self, text):
        self.text = text
        
    def segmentation(self):
        return ViTokenizer.tokenize(self.text)
    
    def split_words(self):
        text = self.segmentation()
        try:
            SPECIAL_CHARACTER = '.,!?;:"\'[]{}()<>@#$%^&*+-/\\|=~`_'
            return [x.strip(SPECIAL_CHARACTER).lower() for x in text.split()]
        except TypeError:
            return []
        
    def covert_unicode(self):
        return re.sub(
            r'à|á|ả|ã|ạ|â|ấ|ầ|ẩ|ẫ|ậ|ă|ắ|ằ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ê|ế|ề|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ô|ố|ồ|ổ|ỗ|ộ|ơ|ớ|ờ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ư|ứ|ừ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Â|Ấ|Ầ|Ẩ|Ẫ|Ậ|Ă|Ắ|Ằ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ê|Ế|Ề|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ô|Ố|Ồ|Ổ|Ỗ|Ộ|Ơ|Ớ|Ờ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ư|Ứ|Ừ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
            lambda x: self.dicchar[x.group()],self.txt
        )
        
    def replace_acronyms(self):
        acronyms = []
        list_text = self.text.split(" ")
        for i in range(len(list_text)):
            for j in range(len(acronyms)):
                if(list_text[i] == acronyms[j][0]):
                    list_text[i] = acronyms[j][1]
        self.text = " ".join(list_text)
        return self.text    
    
    def lowercase(self):
        self.text = self.text.lower()
        return self.text        
    
    def remove_loop_char(self):
        self.text = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), str(self.text), flags=re.IGNORECASE)
        self.text = re.sub(
            r'[^A-Za-zÀ-ỹ\s]',
            ' ', self.text
        )
        return self.text
     
    def remove_stopwords(self):
        stopwords = []
        split_words = self.split_words()
        words = []

        for word in split_words:
            if word not in stopwords:
                words.append(word)

        return ' '.join(words)

In [6]:
import json

def process_json_file(input_file, output_file):
    # Bước 1: Đọc dữ liệu từ file JSON
    with open(input_file, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    # Bước 2: Xử lý từng mục trong dữ liệu JSON, bỏ trường cuối cùng ("time")
    processed_data = []
    for entry in data:
        # Xóa trường cuối cùng "time"
        if "time" in entry:
            del entry["time"]
        
        # Kiểm tra nếu các trường title, subtitle có dữ liệu cần xử lý
        if 'title' in entry:
            processor = DataPreprocessing(entry['title'])
            entry['title'] = processor.split_words()
        
        if 'subtitle' in entry:
            processor = DataPreprocessing(entry['subtitle'])
            entry['subtitle'] = processor.split_words()
        
        processed_data.append(entry)

    # Bước 3: Lưu dữ liệu đã xử lý vào file JSON mới
    with open(output_file, 'w', encoding='utf-8') as json_output_file:
        json.dump(processed_data, json_output_file, ensure_ascii=False, indent=4)

    print(f"Đã xử lý xong và lưu vào file {output_file}")

# Ví dụ sử dụng:
process_json_file('genk_internet.json', 'output.json')


Đã xử lý xong và lưu vào file output.json
